In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import sklearn as skl
from sklearn.decomposition import PCA

#  Import and read the charity_data.csv
CleanedDF = pd.read_csv("Resources/charity_data.csv")
CleanedDF = CleanedDF.drop('EIN', axis=1)
CleanedDF

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...
34294,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [2]:
CleanedDF.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [3]:
# Cleaning Status
CleanedDF = CleanedDF.drop(index=CleanedDF[CleanedDF['STATUS'] == 0].index)
CleanedDF = CleanedDF.drop('STATUS', axis=1)

In [4]:
# Cleaning Special Considerations
CleanedDF = CleanedDF.drop(index=CleanedDF[CleanedDF['SPECIAL_CONSIDERATIONS'] == 'Y'].index)
CleanedDF = CleanedDF.drop('SPECIAL_CONSIDERATIONS', axis=1)

In [5]:
appValue = CleanedDF['APPLICATION_TYPE'].value_counts()

# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(appValue[appValue<1000].index)

# Replace in dataframe
for app in application_types_to_replace:
    CleanedDF['APPLICATION_TYPE'] = CleanedDF['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
CleanedDF['APPLICATION_TYPE'].value_counts()

T3       27006
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [6]:
appClass = CleanedDF['CLASSIFICATION'].value_counts()

# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(appClass[appClass<1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    CleanedDF['CLASSIFICATION'] = CleanedDF['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
CleanedDF['CLASSIFICATION'].value_counts()

C1000    17307
C2000     6069
C1200     4833
Other     2259
C3000     1918
C2100     1882
Name: CLASSIFICATION, dtype: int64

In [7]:
nameValue = CleanedDF['NAME'].value_counts()
nameReplace = list(nameValue[nameValue==1].index)

# Replace in dataframe
for nm in nameReplace:
    CleanedDF['NAME'] = CleanedDF['NAME'].replace(nm,"Other")
    
# Check to make sure binning was successful
CleanedDF['NAME'].value_counts()

Other                                                                 18750
PARENT BOOSTER USA INC                                                 1260
TOPS CLUB INC                                                           765
UNITED STATES BOWLING CONGRESS INC                                      700
WASHINGTON STATE UNIVERSITY                                             492
                                                                      ...  
WASHINGTON EXPLORER SEARCH AND RESCUE                                     2
INTERNATIONAL ALLIANCE THEATRICAL STAGE EMPLOYEES & MOVING PICTURE        2
BIRTH NETWORK                                                             2
NATIONAL ASSOCIATON OF CORPORATE DIRECTORS                                2
NATIONAL SOCIETY COLONIAL DAMES XVII CENTURY                              2
Name: NAME, Length: 792, dtype: int64

In [8]:
IncAmt = CleanedDF['INCOME_AMT'].value_counts()
IncAmt

0                24370
25000-99999       3745
100000-499999     3373
1M-5M              955
1-9999             721
10000-24999        540
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [9]:
# Choose a cutoff value and create a list of application types to be replaced
CleanedDF['INCOME_AMT'] = CleanedDF['INCOME_AMT'].replace(['10000-24999', '1-9999', '25000-99999'], '1-99999')
CleanedDF['INCOME_AMT'] = CleanedDF['INCOME_AMT'].replace(['1M-5M', '5M-10M', '10M-50M', '50M+'], '1M+')
CleanedDF['INCOME_AMT'].value_counts()

0                24370
1-99999           5006
100000-499999     3373
1M+               1519
Name: INCOME_AMT, dtype: int64

In [10]:
CleanedDF.nunique()

NAME                 792
APPLICATION_TYPE       6
AFFILIATION            6
CLASSIFICATION         6
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             4
ASK_AMT             8741
IS_SUCCESSFUL          2
dtype: int64

In [11]:
appDummies = pd.get_dummies(CleanedDF)
appDummies

,ASK_AMT,IS_SUCCESSFUL,NAME_4-H CLUBS & AFFILIATED 4-H ORGANIZATIONS,NAME_AACE INTERNATIONAL,NAME_ACADEMY OF GENERAL DENTISTRY,NAME_ACADEMY OF MANAGED CARE PHARMACY,NAME_ACE MENTOR PROGRAM OF AMERICA INC,NAME_ACTIVE 20-30 UNITED STATES AND CANADA,NAME_ACTS MINISTRY,NAME_ACTS MISSIONS,...,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-99999,INCOME_AMT_100000-499999,INCOME_AMT_1M+
0,5000,1,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0
1,108590,1,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,1,0,0
2,5000,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0
3,6692,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,0,0
4,142590,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0
34295,5000,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0
34296,5000,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,1,0,0,0
34297,5000,1,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0


In [12]:
# Split our preprocessed data into our features and target arrays
x=appDummies.drop('IS_SUCCESSFUL', axis=1).values
y=appDummies['IS_SUCCESSFUL'].values
# Split the preprocessed data into a training and testing dataset
xTrain, xTest, yTrain, yTest = train_test_split(x, y, random_state=34)

In [13]:
# Create scaler instance
scaler = StandardScaler()

# Fit the scaler
X_scaler = scaler.fit(xTrain)

# Scale the data
X_train_scaled = X_scaler.transform(xTrain)
X_test_scaled = X_scaler.transform(xTest)

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(xTrain[0])
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=5, activation='relu'))

# fourth hidden layer
nn.add(tf.keras.layers.Dense(units= 50, activation='relu'))

# fifth hidden layer
nn.add(tf.keras.layers.Dense(units=70, activation='relu'))

# sixth hidden layer
nn.add(tf.keras.layers.Dense(units= 20, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                66000     
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 5)                 155       
                                                                 
 dense_3 (Dense)             (None, 50)                300       
                                                                 
 dense_4 (Dense)             (None, 70)                3570      
                                                                 
 dense_5 (Dense)             (None, 20)                1420      
                                                                 
 dense_6 (Dense)             (None, 1)                 2

In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Create a callback that saves the model's weights every five epochs
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define checkpoints variables
os.makedirs('checkpoints/', exist_ok=True)
checkpoint_path= 'checkpoints/weights.{epoch}.hdf5'

# Create callback 
Callback = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              save_freq='epoch',
                              period=5)

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled, yTrain, epochs=200, callbacks=[Callback])

Epoch 1/200
804/804 [==============================] - 1s 1ms/step - loss: 0.4797 - accuracy: 0.7715
Epoch 2/200
804/804 [==============================] - 1s 970us/step - loss: 0.4340 - accuracy: 0.7905
Epoch 3/200
804/804 [==============================] - 1s 971us/step - loss: 0.4261 - accuracy: 0.7957
Epoch 4/200
804/804 [==============================] - 1s 904us/step - loss: 0.4237 - accuracy: 0.7955
Epoch 5/200
786/804 [============================>.] - ETA: 0s - loss: 0.4232 - accuracy: 0.7964
Epoch 5: saving model to checkpoints\weights.5.hdf5
804/804 [==============================] - 1s 974us/step - loss: 0.4222 - accuracy: 0.7971
Epoch 6/200
804/804 [==============================] - 1s 923us/step - loss: 0.4214 - accuracy: 0.7971
Epoch 7/200
804/804 [==============================] - 1s 907us/step - loss: 0.4217 - accuracy: 0.7982
Epoch 8/200
804/804 [==============================] - 1s 900us/step - loss: 0.4192 - accuracy: 0.7990
Epoch 9/200
804/804 [====================

804/804 [==============================] - 1s 1ms/step - loss: 0.3989 - accuracy: 0.8109
Epoch 65/200
785/804 [============================>.] - ETA: 0s - loss: 0.3986 - accuracy: 0.8107
Epoch 65: saving model to checkpoints\weights.65.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.3988 - accuracy: 0.8106
Epoch 66/200
804/804 [==============================] - 1s 1ms/step - loss: 0.3989 - accuracy: 0.8097
Epoch 67/200
804/804 [==============================] - 1s 1ms/step - loss: 0.3991 - accuracy: 0.8112
Epoch 68/200
804/804 [==============================] - 1s 1ms/step - loss: 0.3988 - accuracy: 0.8110
Epoch 69/200
804/804 [==============================] - 1s 981us/step - loss: 0.3988 - accuracy: 0.8115
Epoch 70/200
781/804 [============================>.] - ETA: 0s - loss: 0.3988 - accuracy: 0.8104
Epoch 70: saving model to checkpoints\weights.70.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.3988 - accuracy: 0.8107
Epoch 71/200
804/80

Epoch 126/200
804/804 [==============================] - 1s 1ms/step - loss: 0.3937 - accuracy: 0.8128
Epoch 127/200
804/804 [==============================] - 1s 1ms/step - loss: 0.3933 - accuracy: 0.8130
Epoch 128/200
804/804 [==============================] - 1s 1000us/step - loss: 0.3933 - accuracy: 0.8127
Epoch 129/200
804/804 [==============================] - 1s 1ms/step - loss: 0.3932 - accuracy: 0.8126
Epoch 130/200
769/804 [===========================>..] - ETA: 0s - loss: 0.3937 - accuracy: 0.8134
Epoch 130: saving model to checkpoints\weights.130.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.3939 - accuracy: 0.8132
Epoch 131/200
804/804 [==============================] - 1s 975us/step - loss: 0.3935 - accuracy: 0.8128
Epoch 132/200
804/804 [==============================] - 1s 938us/step - loss: 0.3930 - accuracy: 0.8131
Epoch 133/200
804/804 [==============================] - 1s 883us/step - loss: 0.3925 - accuracy: 0.8123
Epoch 134/200
804/804 [===

804/804 [==============================] - 1s 1ms/step - loss: 0.3893 - accuracy: 0.8149
Epoch 189/200
804/804 [==============================] - 1s 992us/step - loss: 0.3900 - accuracy: 0.8155
Epoch 190/200
799/804 [============================>.] - ETA: 0s - loss: 0.3899 - accuracy: 0.8158
Epoch 190: saving model to checkpoints\weights.190.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.3899 - accuracy: 0.8157
Epoch 191/200
804/804 [==============================] - 1s 1ms/step - loss: 0.3903 - accuracy: 0.8155
Epoch 192/200
804/804 [==============================] - 1s 1ms/step - loss: 0.3895 - accuracy: 0.8158
Epoch 193/200
804/804 [==============================] - 1s 1ms/step - loss: 0.3894 - accuracy: 0.8161
Epoch 194/200
804/804 [==============================] - 1s 1ms/step - loss: 0.3899 - accuracy: 0.8153
Epoch 195/200
803/804 [============================>.] - ETA: 0s - loss: 0.3903 - accuracy: 0.8149
Epoch 195: saving model to checkpoints\weights.195.

In [18]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled,yTest,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5160 - accuracy: 0.8024 - 286ms/epoch - 1ms/step
Loss: 0.5159851312637329, Accuracy: 0.8023812174797058


In [19]:
# Export our model to HDF5 file
nn.save('h5_files/AlphabetSoupCharity_Optimization.h5')
CleanedDF.to_csv('Resources/optimized.csv')